In [39]:
previousFrame = None
count = 0
test = 0
temp_frame = None
rect = None
usedThresh = None  
globalmask = None

#record videos
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('try13.avi',fourcc, 20.0, (640,480))

if __name__ == '__main__':

    camera =cv2.VideoCapture("try10.avi")  # this for mobil connection cam
    #camera =cv2.VideoCapture(0) 
    #camera =cv2.VideoCapture("http://192.168.0.103:4747/video")
    time.sleep(0.25)
    global previousFrame
    while camera.isOpened():    
        ret, frame = camera.read()
        #Check if the next camera read is not null
        if ret:
            #track hand by motion (dynamically)
            frame, previousFrame = traceHand(frame,previousFrame)
            
            #get the contour for all the white areas in the frame
            _, contours, _ = cv2.findContours(globalmask,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
            
            #get the index of the largest contour
            index = getLargestCountour(contours)

            # if the index is -1 then there was no contours from the begining then move
            if index >=0 and index <= len(contours) -1:
                cnt = contours[index]

                #get the points of the parameter that shape the outer convex
                #false parameter means to retrieve with the hull the convexity  defects (the tie between two fingers)
                hull = cv2.convexHull(cnt,returnPoints = False)

                #get the ties
                defects = cv2.convexityDefects(cnt,hull)
                #draw the defects
                drawDefects(defects)
                # show some windows
                imgsShow()
                
        else:
            print("Video Finished")
            close = False
            while not close:
                key1 = cv2.waitKey(3) & 0xFF
                if key1 == 27 or key1 == ord('q'):
                    close = True

            break
        key = cv2.waitKey(3) & 0xFF
        if key == 27 or key == ord('q'):
            print("Bye")
            break

    camera.release()
    cv2.destroyAllWindows()

Video Finished


In [18]:
import time
import math
import cv2
import numpy as np
import queue 
from matplotlib.pyplot import bar

In [29]:
# track hand by motion
def traceHand(frame,prev):   
    global usedThresh
    kernelErosion = np.ones((1,1),np.uint8) #kernel for erosion
    kernelDilation = np.ones((8,8),np.uint8) #kernel for dilation
    frame = cv2.blur(frame, (9,9)) # blur the original frame "very important for the final image"
    previousFrame = prev



    
    #if it's the first frame just take a copy
    if previousFrame is None:
        previousFrame = frame.copy()
        return frame, previousFrame
    
    # get difference between frames
    frameDiff = cv2.absdiff(previousFrame, frame)    
    # apply binary thresholding on the difference
    thresh = cv2.threshold(frameDiff, 20, 255, cv2.THRESH_BINARY)[1]
    # convert thresholded image to gray-scale
    thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
    # get histogram of thresholded image
    hist = cv2.calcHist([thresh], [0], None, [256], [0,256])
    
    # if white pixels is more than 5000 update newThresh and apply erosion and dialation and threshold it by 250
    if hist[255] >= 5000 :
        previousFrame = frame.copy() #save previous frame
        usedThresh = thresh # important  
        usedThresh = cv2.erode(usedThresh,kernelErosion,iterations =1)
        usedThresh = cv2.dilate(usedThresh, kernelDilation, iterations=5)
        usedThresh = cv2.threshold(usedThresh, 250, 255, cv2.THRESH_BINARY)[1]

    # update newThresh and mask if None
    if usedThresh is None:
            usedThresh = thresh

    #find the contours from the remaining threshold image 
    _, cnts, _ = cv2.findContours(usedThresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    #search for any movement
    frame = searchForMovement(cnts, frame)    

    return frame, previousFrame

In [30]:
def searchForMovement(contours, frame):
    #get the index of the largest contour
    index = getLargestCountour(contours)
    masks = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if index >=0 and index <= len(contours) -1:
        cnt = contours[index]
        M = cv2.moments(cnt) #get some properties from the biggest countour
        cx = int(M['m10']/M['m00']) #x axis centroid
        cy = int(M['m01']/M['m00']) #y axis centroid
        # make a mask with largest contour
        contourmask = np.zeros((frame.shape[0],frame.shape[1]),dtype=np.uint8) 
        cv2.fillPoly(contourmask, pts =[cnt], color=(255,255,255))
        # make a mask with the bounding rectangle of largest contour
        x,y,w,h = cv2.boundingRect(cnt)
        cv2.rectangle(frame,(x,y),(x+w+10,y+h+10),(0,255,0),2)     
        
        rectanglemask = np.zeros((frame.shape[0],frame.shape[1]),dtype=np.uint8)        
        cv2.rectangle(rectanglemask,(x,y),(x+w+10,y+h+10),(255,255,255),-1)

        framehsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # get hue , saturation and value ranges for hand
        hue = framehsv[:,:,0]
        lowHue, highHue = getChanelRange(hue,contourmask)
        
        framehsv[:,:,1] = cv2.blur( framehsv[:,:,1], (9,9))
        sat = framehsv[:,:,1]
        lowSat, highSat = getChanelRange(sat,contourmask)

        framehsv[:,:,2] = cv2.blur( framehsv[:,:,2], (19,19))        
        val = framehsv[:,:,2]  
        gamma = 1.3
        val = gamma_correction(val,gamma)
        lowVal, highVal = getChanelRange(val,contourmask)
        
        # mask framehsv with the rectangle mask
        masked = framehsv.copy()
        masked = cv2.bitwise_and(framehsv,masked,mask = rectanglemask)
        # mask the masked frame with low and high values for each chanel
        masks = mask(masked,lowHue,lowSat,lowVal,highHue,highSat,highVal)
        # calculate histogram of the output inside the rectangle only 
        maskHist = cv2.calcHist([masks],[0], rectanglemask, [256], [0,256])
        global globalmask
        #if white pixels are more than half of black pixels update globalmask
        if globalmask is None or maskHist[255] > 0.5 * maskHist[0]:
            globalmask = masks           
    return frame

In [31]:
#draw the defects
def drawDefects(defects):
    
    if defects is None:
        return
    #put inside each element the tie x,y and the color   true==> red   &  false ==> blue  & start of the finger and change in x and change in y
    vec = []

    #blue refer to reverse hand
    #count number of blue ties
    countDefectsBlue = 0

    #red refer to straight hand
    #count number of red ties
    countDefectsRed = 0
    countDefectsYellow = 0;
    
    try:
        for i in range(defects.shape[0]):

            s,e,f,d = defects[i,0]
            start = tuple(cnt[s][0]) #finger tip
            end = tuple(cnt[e][0]) #another finger tip
            tie = tuple(cnt[f][0]) # tie between them

            #if the distance between start line with end to the tie is less than 1000 this means that this not a tie (noise for example)
            if d < 1000: 
                continue

            cv2.line(frame,start,end,[0,255,0],2)  

            # find length of all sides of triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2) # distance between finger tip and another finger tip
            b = math.sqrt((tie[0] - start[0])**2 + (tie[1] - start[1])**2) # distance between finger tip and a tie
            c = math.sqrt((end[0] - tie[0])**2 + (end[1] - tie[1])**2) # distance between another finger tip and a tie

            # apply cosine rule here
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 180/3.14
            
            #get distance between finger tips in X and Y
            dx = abs(start[0] - end[0])
            dy = abs(start[1] - end[1])

            #if the angle between fingers tips is less than 60 and difference in X is larger than Y it can be W, V, A or M
            if angle <= 60:
                if(dx > dy):                  
                    if abs(tie[1] - start[1])> dy and abs(tie[1] - end[1]) > dy:
                        # if the tie y axis is less than y axis for any of the two finger tips then  the hand is reversed
                        if tie[1] < start[1] or tie[1] <end[1]:
                            countDefectsBlue+= 1 # increment the blue defects
                            vec.append((start,tie,1,end)) # push the tie into the array to print later
                        # if the tie y axis is larger than y axis for any of the two finger tips then  the hand is straight
                        elif tie[1] > start[1] or tie[1] > end[1]:
                            countDefectsRed += 1 # increment the red defects
                            vec.append((start,tie,0,end)) # push the tie into the array to print later
            #if the angle between fingers tips is 90 to 110 it can be L
            elif angle < 110 and angle >=90:
                    # y0 is the finger tip with less Y 
                    y0,y1 =start,end
                    if start[1] < end[1]:
                        y0 = start
                        y1 = end
                    else:
                        y0 = end
                        y1 = start
                    #if the change in x axis between the finger and the tie is less than the shorter finger and the tie
                    if abs(y0[0] - tie[0]) < abs(y1[0] - tie[0]) and abs(y0[1] - tie[1]) > 1.5* abs(y1[1] - tie[1]):   
                        countDefectsYellow += 1 # increment the yellow defects
                        vec.append((start,tie,2,end)) # push the tie into the array to print later

        #print the ties
        printTies(vec,countDefectsRed,countDefectsBlue,countDefectsYellow)  
        #print Characters by count and color of points
        identifyChar(countDefectsRed,countDefectsBlue,countDefectsYellow)    
        
    except Exception as e:
            print(e)

In [32]:
# get range of pixels in chanel
def getChanelRange(chanel,mask):
    # get histogram of chanel inside mask
    Hist = cv2.calcHist([chanel], [0], mask, [256], [0,256])
    # get the range of significant pixel values
    threshold1,threshold2 = getHistPeak(Hist) 
    return threshold1, threshold2

In [34]:
# identify range of  pixel values that has most of pixels
def getHistPeak(hist):
    maxIndex = 0
    c =0
    ci = cj = 0
    i=0
    j =255
    begin = True
    end = 255
    start = 0
    try:
     
        while end - start >20:
            while j > i:
                ci += hist[i]
                cj += hist[j]
                j -=1
                i +=1
            if begin == True:
                begin = False
                c = ci + cj + hist[128]
            if ci >= cj:
                i =start
                j =int((end+start)/2)
                end = j   
            if ci < cj:
                i = int((end+start)/2)
                j =end
                start = i
            ci = 0
            cj = 0
        maxIndex = int((start + end) / 2)
    except Exception as e: 
            print(e)
    startIndex , endIndex = getThreshold(hist,maxIndex,c)
    return startIndex , endIndex

In [35]:
def getThreshold(hist,maxIndex,count):
    threshold1 = maxIndex -1
    threshold2 = maxIndex +1
    p = 0
    while p < 0.65 and threshold2 - threshold1 < 30:
        p1,p2 = 0,0
        if threshold1 > 2:
            p1 = (hist[threshold1] + hist[threshold1-1]+ hist[threshold1-2])/count 
        if threshold2 < 254:
            p2 = (hist[threshold2]+hist[threshold2+1]+hist[threshold2+2])/count
        if p1 > p2 and threshold1 > 2:
            p += (hist[threshold1] + hist[threshold1-1]+ hist[threshold1-2])/count 
            threshold1 -=3
        elif p1 <= p2 and threshold2 < 254:   
            p += (hist[threshold2]+hist[threshold2+1]+hist[threshold2+2])/count
            threshold2  += 3
        if p1 == p2 and p2 == 0:
            break
    return threshold1 , threshold2 

In [36]:
def mask(frame,lowHue,lowSat,lowVal,highHue,highSat,highVal):
   #this is the limit threshold the lowest from the average value and the highest from the average value
    try:
        kernelDilation = np.ones((3,3),np.uint8) #kernel for dilation
        kernelErosion = np.ones((2,2),np.uint8) #kernel for erosion
        
        #get mask for each chanel separately
        lower_blue = np.array([lowHue  ,lowSat - 255 ,lowVal -255])
        upper_blue = np.array([highHue ,highSat + 255,highVal + 255 ])       
        mask1 = cv2.inRange(frame, lower_blue, upper_blue)
        mask1 = cv2.erode(mask1,kernelErosion,iterations =1)
        mask1 = cv2.dilate(mask1,kernelDilation,iterations = 2)
        
        lower_blue = np.array([lowHue - 255  ,lowSat, lowVal -255])
        upper_blue = np.array([highHue + 255 ,highSat +255,highVal + 255 ])      
        mask2 = cv2.inRange(frame, lower_blue, upper_blue)
        
        lower_blue = np.array([lowHue - 255  ,lowSat -255  ,lowVal - 25 ])
        upper_blue = np.array([highHue + 255 ,highSat + 255 ,highVal+25 ])
        mask3 = cv2.inRange(frame, lower_blue, upper_blue)
        mask3 = cv2.dilate(mask3,kernelDilation,iterations = 2)
        
        # get intersection of hue and value channels 
        masks = cv2.bitwise_and(mask1, mask3)
        masks = cv2.erode(masks,kernelErosion,iterations = 1)
        masks = cv2.dilate(masks,kernelDilation,iterations = 2)

        '''
        cv2.namedWindow('hue',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('hue', 800,600)
        cv2.imshow("hue", mask1)
        cv2.namedWindow('val',cv2.WINDOW_NORMAL)
        cv2.resizeWindow('val', 800,600)
        cv2.imshow("val", mask3)
        '''
        
    except Exception as e:
        print(e)
        return frame

    return masks

In [37]:
# apply gamma correction on a frame
def gamma_correction(frame,gamma):
    frame = frame/255
    frame = cv2.pow(frame, gamma)
    frame = np.uint8(frame*255)
    return frame

In [38]:
# returns the count of pixels by histogram
def getPixelsCount(hist):
    count = 0
    for i in range(256):
        count += hist[i]
    return count

In [10]:
#return only  the index of the largest contour
def getLargestCountour(contours):
    largestArea = 0
    index = 0
    for i in range(len(contours)):
        area = cv2.contourArea(contours[i])
        if (area>largestArea):
                largestArea=area
                index= i
    return index

In [11]:
#to print each tie according to the blue and red color and position
def printTies(vec,red,blue,yellow):
    if checkPoints(red,blue,yellow) == False:
        return
    for obj in vec:
        start = obj[0]
        tie = obj[1]
        color = obj[2]
        end = obj[3]
        if color == 0: #red
              cv2.circle(frame,tie,5,[0,0,255],-1)
        elif color == 1: #blue
             cv2.circle(frame,tie,5,[255,0,0],-1)
        elif color == 2: #yellow
             cv2.circle(frame,tie,5,[0,255,255],-1)       
        cv2.line(frame,start,tie,[150,150,150],2)     
        cv2.line(frame,end,tie,[150,150,150],2)
    vec.clear()   

In [12]:
# deciede characters by color and count of points
def identifyChar(red,blue,yellow):
    if checkPoints(red,blue,yellow) == False:
        return
    if red == 1: #v
        showText("V")
    elif red == 2: #w
         showText("W")
    elif blue == 1: #a
         showText("A")
    elif blue == 2: #m
         showText("M")
    elif yellow == 1: #L
         showText("L")

In [13]:
#check if the points are valid
def checkPoints(red,blue,yellow):
    if red > 0:
        if blue > 0 or yellow > 0 or red > 2: 
            return False
    if blue > 0:
        if red > 0 or yellow > 0 or blue > 2:
            return False
    if yellow > 0:
        if red > 0 or blue > 0 or yellow >1:
            return False        
    return True

In [14]:
#print the char on the screen
def showText(string):
    cv2.putText(frame,string, (30, 70), cv2.FONT_HERSHEY_COMPLEX , 4, (50,50,200))

In [15]:
#show images
def imgsShow():
    cv2.namedWindow('frame',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('frame', 800,600)
    cv2.imshow("frame", frame)
    cv2.namedWindow('mask',cv2.WINDOW_NORMAL)
    cv2.resizeWindow('mask', 800,600)
    cv2.imshow("mask", globalmask)
    #out.write(frame)

In [16]:
'''
def getHistPeak(Hist,levelRange):
    PixelsCount = 0
    maxIndex = 0
    maxPixels = 0
    
    count = 0
    for i in range(256):
        count += Hist[i]
        if i+levelRange >255:
            break
        for j in range(i,i+levelRange):
            PixelsCount += Hist[j]
            
        if PixelsCount > maxPixels:
            maxIndex = i
            maxPixels = PixelsCount
        PixelsCount = 0
    startIndex , endIndex = getThreshold(Hist,maxIndex + int(levelRange/2),count)
    return startIndex , endIndex
'''

'\ndef getHistPeak(Hist,levelRange):\n    PixelsCount = 0\n    maxIndex = 0\n    maxPixels = 0\n    \n    count = 0\n    for i in range(256):\n        count += Hist[i]\n        if i+levelRange >255:\n            break\n        for j in range(i,i+levelRange):\n            PixelsCount += Hist[j]\n            \n        if PixelsCount > maxPixels:\n            maxIndex = i\n            maxPixels = PixelsCount\n        PixelsCount = 0\n    startIndex , endIndex = getThreshold(Hist,maxIndex + int(levelRange/2),count)\n    return startIndex , endIndex\n'